<a href="https://colab.research.google.com/github/acdc2019/algo-trading/blob/main/python/notebooks/strategies/rsi_strategy/RSIStrategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **RSI Strategy**
## **15 mins strategy**
* Step 1: A 15 min candle has to close above 70 RSI value.
* Step 2: Above High of this candle Buy signal is generated.
* Step 3: Stop Loss of the trade has to be below the earlier 15 min candles low.
* Step 4: Volume of the candle in which buy signal is generated has to be atleast 2 to 3 times of earlier 5 candles (***Q: average of earlier 5 candle volume?***)
* Step 5: Hourly candles RSI should be above 50 for confirmation.
* Step 6: In the last 5 candles the stock movement should not be more than 4% of the day movement
* Step 7: If Stop Loss of the candle is less than 6000 INR then the trade should be executed

#### **Strategy Parameters**
* window_start, window_end: Dates between which to look for signal
* rsi_15min = 70
* rsi_60min = 50
* stop_loss = 6000
* back_candles = 5
* volume_multiple = 2 or 3 times
* daily_movement_pct = 4

Install libs

In [14]:
!pip install ta
!pip install pandas==1.3.5
!pip install plotly
!pip install numpy

In [15]:
!wget https://raw.githubusercontent.com/acdc2019/algo-trading/main/python/strategy/helpers.py?token=GHSAT0AAAAAABQZJYESIVLFFUHVXAVEXEWCYPMANAQ -O helpers.py

--2022-01-22 13:50:15--  https://raw.githubusercontent.com/acdc2019/algo-trading/main/python/strategy/helpers.py?token=GHSAT0AAAAAABQZJYESIVLFFUHVXAVEXEWCYPMANAQ
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2022-01-22 13:50:15 ERROR 404: Not Found.



Import Libs

In [16]:
import helpers
import pandas as pd
from ta.momentum import RSIIndicator
from plotly.subplots import make_subplots
import numpy as np

In [17]:
def get_tanaji_pct(df, index, n, high):
  prev_candles = helpers.get_previous_candles(df, index, n)
  min_low = prev_candles['Low'].min()
  daily_movement = high - min_low
  tanaji_pct = (daily_movement/min_low)*100
  return tanaji_pct

def get_hourly_rsi(df_15min, index):
  df_15min_temp = df_15min[:index]
  #print(index)
  #print(df_15min_temp.tail(10))
  df_60min_o = df_15min_temp['Open'].resample('60Min', offset='30Min').apply({'Open': 'first'})
  df_60min_h = df_15min_temp['High'].resample('60Min', offset='30Min').apply({'High': 'max'})
  df_60min_l = df_15min_temp['Low'].resample('60Min', offset='30Min').apply({'Low': 'min'})
  df_60min_c = df_15min_temp['Close'].resample('60Min', offset='30Min').apply({'Close': 'last'})
  df_60min_vol = df_15min_temp['Volume'].resample('60Min', offset='30Min').apply({'Volume': 'sum'})
  
  df_60min = pd.concat([df_60min_o, df_60min_h, df_60min_l, df_60min_c, df_60min_vol], axis=1)
  df_60min.dropna(subset=['Open'], inplace=True)

  rsi = RSIIndicator(df_60min['Close']).rsi()
  df_60min = df_60min.assign(rsi=rsi.values)
  return df_60min

## Load 15 min and 60 min stock data and calculate RSI

In [64]:
file_15min = '/content/SBIN-HIST.csv'

df_15min = pd.read_csv(file_15min, parse_dates=['Date'], index_col=['Date'])
rsi = RSIIndicator(df_15min['Close']).rsi()
df_15min = df_15min.assign(rsi=rsi.values)
df_15min.tail()


,Open,High,Low,Close,Volume,rsi
Date,,,,,,
2021-12-17 14:15:00+05:30,470.40,470.75,468.10,469.00,552209,30.640452
2021-12-17 14:30:00+05:30,469.00,469.50,468.20,468.60,336276,29.662491
2021-12-17 14:45:00+05:30,468.45,468.50,467.50,468.00,504704,28.208115
2021-12-17 15:00:00+05:30,468.25,469.40,466.60,467.55,1390599,27.133577
2021-12-17 15:15:00+05:30,467.60,468.50,466.85,468.30,1697650,31.796798


## Set Strategy Parameters

In [65]:
window_start = '2021-01-01 00:00:00'
window_end = '2021-12-15 00:00:00'
rsi_15min = 70
rsi_60min = 50
lot_size = 1500
stop_loss = 6000
back_candles = 5
volume_multiple = 2 # 2 or 3 times
daily_movement_pct = 4


## Actual Strategy Implementation

### Step 1: A 15 min candle has to close above 70 RSI value
Get all the candles in the window that close above required RSI value

In [66]:
curr_window_df = df_15min[window_start:window_end]

rsi_filter = (curr_window_df['rsi'] > rsi_15min)
df = curr_window_df[rsi_filter]

bullish_filter = (df['Close'] > df['Open'])
df = df[bullish_filter]

temp_df = pd.DataFrame(columns=['Open','High','Low','Close','Volume','rsi'])
temp_df.index.name = 'Date'
back_candles_df = pd.DataFrame()

for index, row in df.iterrows():
  back_candles_df = helpers.get_previous_candles(curr_window_df, index, 1)
  prev_rsi = 0
  if (back_candles_df.shape[0] != 0):
    prev_rsi = back_candles_df.iloc[0]['rsi']

  if (prev_rsi < 70):
    temp_df.loc[index] = row

df = temp_df[['Open','High','Low','Close','Volume','rsi']]
df.head()


,Open,High,Low,Close,Volume,rsi
Date,,,,,,
2021-01-04 09:15:00+05:30,281.85,283.15,281.40,283.00,3641399.0,80.043139
2021-01-06 09:15:00+05:30,283.00,285.85,282.10,285.80,3777614.0,78.347796
2021-01-12 12:15:00+05:30,286.95,289.15,286.85,288.50,3151726.0,73.412312
2021-01-12 14:45:00+05:30,289.25,291.40,289.20,290.80,4148455.0,71.911874
2021-01-13 14:15:00+05:30,298.90,303.80,298.50,303.65,6434120.0,73.715822



### Step 4: Volume check

In [67]:
temp_df = pd.DataFrame(columns=['Open','High','Low','Close','Volume','rsi','MeanVolume'])
temp_df.index.name = 'Date'
back_candles_df = pd.DataFrame()
for index, row in df.iterrows():
  # Get mean volume of previous candles
  back_candles_df = helpers.get_previous_candles(curr_window_df, index, back_candles)
  mean_volume = back_candles_df['Volume'].mean()
  row['MeanVolume'] = mean_volume

  # Compare volume against previous mean volume
  if(row['Volume'] > volume_multiple*mean_volume):
    # print('Current Volume:', row['Volume'], 'greather than 2 times mean volume', mean_volume)
    temp_df.loc[index] = row

df = temp_df[['Open','High','Low','Close','Volume','rsi']]
temp_df.head()

,Open,High,Low,Close,Volume,rsi,MeanVolume
Date,,,,,,,
2021-01-04 09:15:00+05:30,281.85,283.15,281.40,283.00,3641399.0,80.043139,1361650.8
2021-01-06 09:15:00+05:30,283.00,285.85,282.10,285.80,3777614.0,78.347796,1363188.6
2021-01-12 12:15:00+05:30,286.95,289.15,286.85,288.50,3151726.0,73.412312,1562428.0
2021-01-12 14:45:00+05:30,289.25,291.40,289.20,290.80,4148455.0,71.911874,1058050.4
2021-01-13 14:15:00+05:30,298.90,303.80,298.50,303.65,6434120.0,73.715822,2185024.2


### Step 5: Hourly candle RSI check

In [68]:
temp_df = pd.DataFrame(columns=df.columns)
temp_df.index.name = 'Date'

for index, row in df.iterrows():
  df_60min = get_hourly_rsi(df_15min, index)
  #print(df_60min.tail(15))
  
  rsi = df_60min.iloc[-1]['rsi']
  if(rsi > rsi_60min):
    # This 15min candle is eligible for signal
    print('Hourly candle RSI is greater than 50..', rsi)
    temp_df.loc[index] = row

df = temp_df
df.head()


Hourly candle RSI is greater than 50.. 73.31156059790013
Hourly candle RSI is greater than 50.. 68.9697861795206
Hourly candle RSI is greater than 50.. 60.64876282096961
Hourly candle RSI is greater than 50.. 63.75504934774057
Hourly candle RSI is greater than 50.. 76.48490778258457
Hourly candle RSI is greater than 50.. 79.68673872531157
Hourly candle RSI is greater than 50.. 59.40484640823018
Hourly candle RSI is greater than 50.. 71.3477245886101
Hourly candle RSI is greater than 50.. 81.16432794666062
Hourly candle RSI is greater than 50.. 85.69999097287034
Hourly candle RSI is greater than 50.. 61.15095189111927
Hourly candle RSI is greater than 50.. 64.48324227967987
Hourly candle RSI is greater than 50.. 68.82257795134606
Hourly candle RSI is greater than 50.. 65.83666930774736
Hourly candle RSI is greater than 50.. 55.44334483057802
Hourly candle RSI is greater than 50.. 51.709857939432965
Hourly candle RSI is greater than 50.. 55.2408388159516
Hourly candle RSI is greater than

,Open,High,Low,Close,Volume,rsi
Date,,,,,,
2021-01-04 09:15:00+05:30,281.85,283.15,281.40,283.00,3641399.0,80.043139
2021-01-06 09:15:00+05:30,283.00,285.85,282.10,285.80,3777614.0,78.347796
2021-01-12 12:15:00+05:30,286.95,289.15,286.85,288.50,3151726.0,73.412312
2021-01-12 14:45:00+05:30,289.25,291.40,289.20,290.80,4148455.0,71.911874
2021-01-13 14:15:00+05:30,298.90,303.80,298.50,303.65,6434120.0,73.715822


### Step 6: Stock movement check

In [69]:
temp_df = pd.DataFrame(columns=df.columns)
temp_df.index.name = 'Date'

for index, row in df.iterrows():
  tanaji_pct = get_tanaji_pct(df_15min, index, 5, row['High'])
  print('tanaji_pct:', tanaji_pct)
  if(tanaji_pct < daily_movement_pct):
    temp_df.loc[index] = row

df = temp_df
temp_df

tanaji_pct: 1.5602582496413075
tanaji_pct: 1.9436519258202734
tanaji_pct: 2.3358697575650207
tanaji_pct: 2.156003505696749
tanaji_pct: 3.105379263533017
tanaji_pct: 1.518615284128032
tanaji_pct: 2.8520499108734403
tanaji_pct: 5.0977653631285005
tanaji_pct: 2.209043736100812
tanaji_pct: 4.4575381068272
tanaji_pct: 1.5108834827144628
tanaji_pct: 2.1656050955414012
tanaji_pct: 2.08932816214188
tanaji_pct: 2.6054590570719602
tanaji_pct: 2.47828308635668
tanaji_pct: 3.082283570088448
tanaji_pct: 2.4616745283018937
tanaji_pct: 3.4283102239829346
tanaji_pct: 3.88422315536894
tanaji_pct: 2.3941323000276715
tanaji_pct: 2.433503112620267
tanaji_pct: 2.771241830065365
tanaji_pct: 1.3563049853372462
tanaji_pct: 1.0812109562710237
tanaji_pct: 2.229146692233943
tanaji_pct: 1.8095801301005268
tanaji_pct: 1.093210586881473
tanaji_pct: 1.451820629902819
tanaji_pct: 2.1038711228660736
tanaji_pct: 0.9988249118683901
tanaji_pct: 1.379391652912041
tanaji_pct: 1.0837897680922917
tanaji_pct: 1.75276752767526

,Open,High,Low,Close,Volume,rsi
Date,,,,,,
2021-01-04 09:15:00+05:30,281.85,283.15,281.40,283.00,3641399.0,80.043139
2021-01-06 09:15:00+05:30,283.00,285.85,282.10,285.80,3777614.0,78.347796
2021-01-12 12:15:00+05:30,286.95,289.15,286.85,288.50,3151726.0,73.412312
2021-01-12 14:45:00+05:30,289.25,291.40,289.20,290.80,4148455.0,71.911874
2021-01-13 14:15:00+05:30,298.90,303.80,298.50,303.65,6434120.0,73.715822
2021-01-15 09:15:00+05:30,306.80,310.85,306.45,310.55,3917485.0,71.129275
2021-01-29 14:15:00+05:30,284.50,288.50,284.40,287.65,4747330.0,71.312687
2021-02-04 13:30:00+05:30,338.80,344.70,331.45,343.25,18455133.0,70.831028
2021-02-12 12:45:00+05:30,393.40,396.40,393.25,396.00,3277277.0,70.028053


### Step 7: Stop Loss check

In [70]:
temp_df = pd.DataFrame(columns=df.columns)
temp_df.index.name = 'Date'

for index, row in df.iterrows():
  prev_candle = helpers.get_previous_candles(df_15min, index, 1)
  prev_low = prev_candle.iloc[0]['Low']
  if((row['High'] - prev_low)*lot_size > stop_loss):
    print(index, 'Stop Loss greater than 6000 INR. Do not trade', (row['High'] - prev_low)*lot_size)
  else:
    print(index, 'Stop Loss within range', (row['High'] - prev_low)*lot_size)
    temp_df.loc[index] = row

df = temp_df
df

2021-01-04 09:15:00+05:30 Stop Loss greater than 6000 INR. Do not trade 6149.999999999949
2021-01-06 09:15:00+05:30 Stop Loss greater than 6000 INR. Do not trade 6900.000000000035
2021-01-12 12:15:00+05:30 Stop Loss within range 5625.0
2021-01-12 14:45:00+05:30 Stop Loss greater than 6000 INR. Do not trade 6824.999999999932
2021-01-13 14:15:00+05:30 Stop Loss greater than 6000 INR. Do not trade 13725.000000000051
2021-01-15 09:15:00+05:30 Stop Loss greater than 6000 INR. Do not trade 6075.000000000017
2021-01-29 14:15:00+05:30 Stop Loss greater than 6000 INR. Do not trade 9974.999999999965
2021-02-04 13:30:00+05:30 Stop Loss greater than 6000 INR. Do not trade 9750.0
2021-02-12 12:45:00+05:30 Stop Loss greater than 6000 INR. Do not trade 6750.0
2021-02-15 09:15:00+05:30 Stop Loss greater than 6000 INR. Do not trade 12450.000000000016
2021-02-15 14:00:00+05:30 Stop Loss greater than 6000 INR. Do not trade 10500.0
2021-02-25 09:15:00+05:30 Stop Loss greater than 6000 INR. Do not trade 11

,Open,High,Low,Close,Volume,rsi
Date,,,,,,
2021-01-12 12:15:00+05:30,286.95,289.15,286.85,288.50,3151726.0,73.412312
2021-06-03 13:30:00+05:30,437.45,439.25,437.20,439.15,2542980.0,71.056737
2021-06-10 14:30:00+05:30,430.70,433.25,430.60,432.60,2866563.0,71.437890
2021-08-03 13:00:00+05:30,440.70,443.25,440.40,443.10,2174030.0,76.272517
2021-08-30 14:00:00+05:30,419.50,420.85,419.25,420.55,1048109.0,71.185571
2021-09-14 09:30:00+05:30,434.90,436.75,434.85,435.95,1200203.0,73.631323
2021-09-15 13:30:00+05:30,437.50,439.65,437.40,439.55,752754.0,71.943083
2021-09-29 15:00:00+05:30,457.60,461.05,457.35,460.60,8683791.0,74.919167
2021-10-04 14:00:00+05:30,461.25,463.50,460.60,462.75,1172120.0,71.857061


## **Final Signal**

In [71]:
for index, row in df.iterrows():
  prev_candle = helpers.get_previous_candles(df_15min, index, 1)
  stop_loss = prev_candle.iloc[0]['Low']
  print('For candle:', index, ', rsi:', row['rsi'], 'Entry Price', row['High'], 'Stop Loss:', stop_loss)

For candle: 2021-01-12 12:15:00+05:30 , rsi: 73.4123123441522 Entry Price 289.15 Stop Loss: 285.4
For candle: 2021-06-03 13:30:00+05:30 , rsi: 71.05673662648526 Entry Price 439.25 Stop Loss: 435.5
For candle: 2021-06-10 14:30:00+05:30 , rsi: 71.43788993379513 Entry Price 433.25 Stop Loss: 430.0
For candle: 2021-08-03 13:00:00+05:30 , rsi: 76.27251689958327 Entry Price 443.25 Stop Loss: 440.3
For candle: 2021-08-30 14:00:00+05:30 , rsi: 71.18557131430015 Entry Price 420.85 Stop Loss: 418.9
For candle: 2021-09-14 09:30:00+05:30 , rsi: 73.63132290261997 Entry Price 436.75 Stop Loss: 434.1
For candle: 2021-09-15 13:30:00+05:30 , rsi: 71.94308328750505 Entry Price 439.65 Stop Loss: 437.4
For candle: 2021-09-29 15:00:00+05:30 , rsi: 74.91916725836418 Entry Price 461.05 Stop Loss: 457.2
For candle: 2021-10-04 14:00:00+05:30 , rsi: 71.85706078154344 Entry Price 463.5 Stop Loss: 459.7
For candle: 2021-10-12 14:00:00+05:30 , rsi: 71.40080713715538 Entry Price 480.5 Stop Loss: 476.55
For candle: 

In [72]:
import plotly.graph_objects as go
import plotly.offline as py

back=5
next=50
sig_param_col1 = ['Time','Entry INR','StopLoss INR','Daily Mov %','Volume','Mean Vol','15Min RSI','Hourly RSI']
for index, row in df.iterrows():
  prev_candles = helpers.get_previous_candles(df_15min, index, back, True)
  next_candles = helpers.get_next_candles(df_15min, index, next)
  
  candles = pd.concat([prev_candles, next_candles])
  candles['DateStr'] = candles.index.strftime('%d-%m %H:%M')

  # Get all strategy params for this signal
  stop_loss_candle = helpers.get_previous_candles(df_15min, index, 1)
  vol_back_candles_df = helpers.get_previous_candles(df_15min
                                                 , index, back_candles)
  sig_stop_loss = stop_loss_candle.iloc[0]['Low']
  sig_entry = candles['High'][back]
  sig_daily_mov_pct = np.round(get_tanaji_pct(df_15min, index, 5, row['High']),2)
  sig_rsi = np.round(candles['rsi'][back],2)
  sig_hourly_rsi = np.round(get_hourly_rsi(df_15min, index).iloc[-1]['rsi'],2)
  sig_mean_volume = vol_back_candles_df['Volume'].mean()
  sig_volume = row['Volume']

  fig = make_subplots(rows=2, cols=2, shared_xaxes=False,
               subplot_titles=('OHLC', 'Signal Params', 'Volume & RSI',''), 
               vertical_spacing=0.1, 
               horizontal_spacing=0.01,
               row_width=[0.5, 0.7],
               column_widths=[0.8,0.2],
               specs=[[{"secondary_y": False, "type": "candlestick"},{"secondary_y": False, "type":"table"}], 
                      [{"secondary_y": True}, {"secondary_y": False}]])
    
  fig.add_trace(go.Candlestick(x=candles['DateStr'],
                     open=candles['Open'],
                     high=candles['High'],
                     low=candles['Low'],
                     close=candles['Close'],
                     name='Signal Chart',
                     increasing_line_color='yellow',
                     increasing_fillcolor='yellow',
                     decreasing_line_color='red',
                     decreasing_fillcolor='red',),
                     row=1,col=1)
  
  fig.add_annotation(x=back,y=sig_entry,
                     text='Signal')
  
  # Position Entry Point
  fig.add_shape(type='line', 
                x0=-1,x1=back+next+2,
                y0=sig_entry, y1=sig_entry, 
                line=dict(color='Green'),
                row=1,col=1)
  
  # Position Stop Loss
  fig.add_shape(type='line', 
                x0=-1,x1=back+next+2,
                y0=sig_stop_loss, y1=sig_stop_loss, 
                line=dict(color='Red'),
                row=1,col=1)

  # Signal Parameters Table
  fig.add_trace(go.Table(header=dict(values=['Param','Value'],
                                     line_color='white',
                                     fill_color='darkslategray',
                                     align='left'),
                         cells=dict(values=[sig_param_col1,
                                           [index.strftime('%d-%m %H:%M'), sig_entry, 
                                            sig_stop_loss, sig_daily_mov_pct, sig_volume, 
                                            sig_mean_volume, sig_rsi, sig_hourly_rsi]],
                                    line_color='white',
                                    fill_color='black',
                                    align='left')),
                row=1, col=2)

  # RSI
  fig.add_trace(go.Scatter(x=candles['DateStr'], y=candles['rsi'], name='rsi',            
           marker_color='Cyan'),
           row=2, col=1)
  
  # Volume
  fig.add_trace(go.Bar(x=candles['DateStr'], y=candles['Volume'], name='Volume',            
           marker_color='rgb(55, 55, 109)',
           width=np.array([0.5]*candles.size)),
           secondary_y=True,
          row=2, col=1)
  
  fig.add_annotation(x=back,y=sig_rsi,
                     text='Signal',row=2, col=1)
  
  fig.add_shape(type='line', x0=-1,x1=back+next+2,y0=rsi_15min, y1=rsi_15min, 
                line=dict(color='Green'),
                row=2,col=1)

  fig.update_xaxes(type='category', rangeslider=dict(visible=False))
  fig.update_xaxes(showgrid=False, nticks=5)
  fig.update_yaxes(showgrid=False)
  fig.update_layout(
    title='Signal generated for RSI 15mins Strategy',
    title_x = 0.5,
    autosize=False,
    width=1450,
    height=750,
    plot_bgcolor='rgb(5,5,5)',
    paper_bgcolor='rgb(0,0,0)',
    font_color='white')

  py.iplot(fig)
  